# Audio Sample to Sega Genesis Synth Path Estimator - Algorithm 7

In [ ]:
import numpy as np
import os
import pandas as pd
import pickle as pk
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential, model_from_json # Will experiment with various architectures
from tensorflow.keras.layers import Dense

# Side note: don't actually run this in Jupyter. This is simply a whiteboard for the model design.
# Transfer to a .py file, and run this on Linux with AMD GPU support.

SETUP

In [ ]:
#open directory with the spectrograms, raw audio data (if I used it), and register data in it.
used_raw_data = True

os.chdir('..')
os.chdir('./data/')

RegisterTargets = pd.read_csv('SampleRegisters7.csv', index_col = 0)
print(len(RegisterTargets),'patches created with algorithm 7 found.')

SampleFeatures = pd.read_csv('SampleSpectra7.csv', index_col = 0).drop('Time') # No longer need the Time subcolumn names, as the main column labels incorporated this info just fine.
SampleFeatures['116.204'] = SampleFeatures['116.204'].fillna(0) # For some reason, the last column had NaNs near the bottom.

if used_raw_data:
    SampleWav = pd.read_csv('SampleWav7.csv', index_col = 0)
    SampleFeatures = pd.concat([SampleFeatures, SampleWav], axis = 1, join = 'inner')
    del SampleWav

os.chdir('..')
os.chdir('./models/')
pca = pk.load(open('pca7.pkl','rb'))
ReducedFeatures = pca.transform(SampleFeatures)
del SampleFeatures

np.shape(ReducedFeatures)

for DTcol in ['DT1', 'DT2', 'DT3', 'DT4']:
    RegisterTargets[DTcol] = RegisterTargets[DTcol] + 8 # Make this non-zero for ReLU.

RegisterTargets.head()

TRAINING AN ESTIMATOR MODEL

In [ ]:
model7 = Sequential()
model7.add(Dense(220, input_shape = (200,), activation='relu'))
model7.add(Dense(200, activation='relu'))
model7.add(Dense(180, activation='relu'))
model7.add(Dense(160, activation='relu'))
model7.add(Dense(140, activation='relu'))
model7.add(Dense(120, activation='relu'))
model7.add(Dense(100, activation='relu'))
model7.add(Dense(80, activation='relu'))
model7.add(Dense(60, activation='relu'))
model7.add(Dense(37, activation='relu')) # 41 numerical outputs, but only 37 if we exclude release.

# To do: experiment with these options.
model7.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mean_squared_error'])

model7.fit(ReducedFeatures,RegisterTargets,epochs = 150, batch_size = 20)

# To do: determine more insightful metrics.
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy))

SAVING THE MODEL TO JSON TO BE IMPORTED INTO MAIN PROGRAM

In [ ]:
os.chdir('..')
os.chdir('./models/')

# serialize model to JSON
model_json = model7.to_json()
with open("model7.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model7.save_weights("model7.h5")
print("Saved model to disk")